## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
import numpy as np
from numba import njit
from numba import jit
N = 1000000
A = np.random.randint(0, 1000, size=N)
@njit
def add_100(arr):
    B = arr + 100
    return B.mean()
mean_B = add_100(A)
print("Среднее значение массива B :", mean_B)

Среднее значение массива B : 599.391941


In [3]:
B = A + 100
B.mean()

599.391941

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [4]:
import pandas as pd
import random
N = 2000000
df = pd.DataFrame(np.random.randn(N, 4), columns = list('ABCD'))
lett = list('abcdefghijklmnopqrstuvwxyz')
df['key'] = [random.choice(lett) for _ in range(N)]
res = df[df['key'].str[:5].isin(['a', 'b', 'c', 'd', 'e'])]
res

,A,B,C,D,key
0,-0.743748,-0.129055,-0.269557,-1.211358,c
5,0.923673,0.105935,1.574308,0.332442,b
7,-0.212889,0.852510,1.522510,0.013203,b
8,0.081452,0.084678,-0.694534,-0.286888,c
9,0.176226,0.915990,-2.024433,0.584732,e
...,...,...,...,...,...
1999973,0.412246,1.453274,0.470495,0.475223,b
1999979,0.100176,2.169614,-2.014760,-1.248080,c
1999993,-0.064247,-0.351589,0.878765,-1.262707,d
1999995,0.669758,-0.355146,1.313168,0.499732,a


## Лабораторная работа 3

In [ ]:
!pip install line_profiler

In [6]:
import line_profiler

In [7]:
%load_ext line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [24]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv', parse_dates = ['date'])

In [25]:
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [27]:
reviews

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [10]:
import timeit
import time
def mean_A():
    c = 0
    summa = 0
    for i, r in reviews.iterrows():
        if r['date'].year == 2010:
            summa += r['rating']
            c += 1
    return summa/c
def mean_B():
    df = reviews[reviews['date'].dt.year == 2010]
    c = 0
    summa = 0
    for i, r in df.iterrows():
        summa += r['rating']
        c += 1
    return summa/c
def mean_C():
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()
print(f'Пункт A: {mean_A()},Пункт B: {mean_B()}, Пункт C: {mean_C()}')

Пункт A: 4.476252723311547,Пункт B: 4.476252723311547, Пункт C: 4.476252723311547


In [36]:
%timeit mean_A()
%timeit mean_B()
%timeit mean_C()

5.18 s ± 136 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
489 ms ± 13.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
12.1 ms ± 33.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [11]:
%lprun -f mean_A mean_A()

Timer unit: 1e-09 s

Total time: 1.47846 s
File: /tmp/ipykernel_170/1150550080.py
Function: mean_A at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def mean_A():
     4         1        655.0    655.0      0.0      c = 0
     5         1        228.0    228.0      0.0      summa = 0
     6     23239 1265204786.0  54443.2     85.6      for i, r in reviews.iterrows():
     7     20944  194816650.0   9301.8     13.2          if r['date'].year == 2010:
     8      2295   17670216.0   7699.4      1.2              summa += r['rating']
     9      2295     767384.0    334.4      0.1              c += 1
    10         1        902.0    902.0      0.0      return summa/c

In [12]:
%lprun -f mean_B mean_B()

Timer unit: 1e-09 s

Total time: 0.142072 s
File: /tmp/ipykernel_170/1150550080.py
Function: mean_B at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           def mean_B():
    12         1    5332389.0 5332389.0      3.8      df = reviews[reviews['date'].dt.year == 2010]
    13         1        433.0    433.0      0.0      c = 0
    14         1        276.0    276.0      0.0      summa = 0
    15      2295  114831200.0  50035.4     80.8      for i, r in df.iterrows():
    16      2295   21283090.0   9273.7     15.0          summa += r['rating']
    17      2295     623955.0    271.9      0.4          c += 1
    18         1        867.0    867.0      0.0      return summa/c

In [13]:
%lprun -f mean_C mean_C()

Timer unit: 1e-09 s

Total time: 0.00451594 s
File: /tmp/ipykernel_170/1150550080.py
Function: mean_C at line 19

Line #      Hits         Time  Per Hit   % Time  Line Contents
    19                                           def mean_C():
    20         1    4515942.0 4515942.0    100.0      return reviews[reviews['date'].dt.year == 2010]['rating'].mean()

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [26]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
get_word_reviews_count(reviews)

{'Last': 15,
 'week': 138,
 'whole': 986,
 'sides': 51,
 'of': 19894,
 'frozen': 506,
 'salmon': 136,
 'fillet': 22,
 'was': 16322,
 'on': 6286,
 'sale': 31,
 'in': 11428,
 'my': 7972,
 'local': 117,
 'supermarket,': 3,
 'so': 8278,
 'I': 52443,
 'bought': 243,
 'tons': 29,
 '(okay,': 2,
 'only': 2577,
 '3,': 7,
 'but': 7909,
 'total': 75,
 'weight': 21,
 'over': 1739,
 '10': 426,
 'pounds).': 1,
 '': 39383,
 'This': 7304,
 'recipe': 7615,
 'is': 10073,
 'perfect': 819,
 'for': 22311,
 'fillet,': 3,
 'even': 1446,
 'though': 427,
 'it': 20185,
 'calls': 96,
 'steaks.': 19,
 'cut': 1235,
 'up': 2494,
 'the': 49056,
 'into': 1275,
 'individual': 63,
 'portions': 33,
 'and': 40287,
 'followed': 945,
 'instructions': 142,
 'exactly.': 115,
 "I'm": 1349,
 'one': 2849,
 'those': 438,
 'food': 432,
 'combining': 6,
 'diets,': 3,
 'left': 862,
 'out': 4347,
 'white': 618,
 'wine': 243,
 'added': 4024,
 'just': 4540,
 'a': 30594,
 'dash': 110,
 'vinegar': 234,
 'instead': 1894,
 '(just': 43,
 '

In [17]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-09 s

Total time: 5.3971 s
File: /tmp/ipykernel_170/3215794765.py
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count(df):
     2         1        947.0    947.0      0.0      word_reviews = {}
     3     23234 1367315325.0  58849.8     25.3      for _, row in df.dropna(subset=['review']).iterrows():
     4     23234  396361246.0  17059.5      7.3          recipe_id, review = row['recipe_id'], row['review']
     5     23234   95984599.0   4131.2      1.8          words = review.split(' ')
     6   1247435  248970591.0    199.6      4.6          for word in words:
     7   1190227  349414259.0    293.6      6.5              if word not in word_reviews:
     8     57208   19942220.0    348.6      0.4                  word_reviews[word] = []
     9   1247435  466089330.0    373.6      8.6              word_reviews[word].append(recipe_id)
    10 

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [30]:
reviews = pd.read_csv('reviews_sample.csv')

In [32]:
df4 = reviews[['recipe_id', 'rating']].dropna()
df4 = df4[df4['rating'] != 0]
df4 = pd.merge(df4, df4.groupby(['recipe_id'])['rating'].mean().rename('mean_rating'), how='left', left_on='recipe_id', right_index=True)
df4

,recipe_id,rating,mean_rating
0,57993,5,4.818182
1,142201,5,5.000000
2,252013,4,4.000000
3,404716,5,4.555556
4,129396,5,5.000000
...,...,...,...
126690,314698,5,5.000000
126691,335534,4,3.833333
126693,222001,5,5.000000
126694,354979,5,4.064516


In [33]:
df4_a = df4['rating'].to_numpy()
df4_b = df4['mean_rating'].to_numpy()
def mape_a():
    n = len(df4_a)
    return sum([abs(df4_a[i] - df4_b[i]) / df4_a[i] for i in range(n)]) / n * 100
mape_a()

11.17155025905884

In [34]:
%timeit mape_a()

333 ms ± 7.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
@jit(nopython = True)
def mape_b():
    n = len(df4_a)
    return sum([abs(df4_a[i] - df4_b[i]) / df4_a[i] for i in range(n)]) / n * 100
mape_b()

11.17155025905884

In [36]:
%timeit mape_b()

822 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [37]:
def mape_c():
    return np.mean(np.fabs(df4_a - df4_b) / df4_a) * 100
mape_c()

11.171550259058085

In [38]:
%timeit mape_c()

737 µs ± 5.14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [39]:
@jit(nopython = True)
def mape_d():
    return np.mean(np.fabs(df4_a - df4_b) / df4_a) * 100
mape_d()

11.17155025905884

In [40]:
%timeit mape_d()

291 µs ± 315 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
